In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set(rc={'image.cmap': 'coolwarm'})

#from numba import jit,prange

import time
import os

SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 20

plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [66]:
def remove_triangle(df):
    # Remove triangle of a symmetric matrix and the diagonal
    
    df = df.astype(float)
    df.values[np.triu_indices_from(df, k=1)] = np.nan
    df  = ((df.T).values.reshape((1,(df.shape[0])**2)))
    df = df[~np.isnan(df)]
    df = df[df!=1]
    return (df).reshape((1,len(df)))

# Ingestion

In [67]:
path = r'/Users/rodrigo/Post-Grad/CC400/ADHD200_CC400_TCs_filtfix'
phenotypic = pd.DataFrame()
files = os.listdir(path)
files.sort()

phenotypic = pd.DataFrame()
motion = pd.DataFrame()
df = pd.DataFrame()


# Dataset LOOP
for dataset in files[1:-1]:
    print(dataset)
    path = r'/Users/rodrigo/Post-Grad/CC400/ADHD200_CC200_TCs_filtfix'
    path = os.path.join(path,dataset)
    files = os.listdir(path)
    files.sort()
    start = time.time()
    
    # Subject LOOP
    for subject in files[1:]:
        print(subject)
        
        # Saving phenotypic data
        if str(subject)[-3:] == 'csv':
            
            if str(subject)[-14:] == 'phenotypic.csv':
                df_aux = pd.read_csv(os.path.join(path,subject))
                df_aux[subject[:-15]] = str(subject[:-15])
                df_aux = df_aux.set_index([subject[:-15], 'ScanDir ID'], inplace=False)
                phenotypic = pd.concat([df_aux,phenotypic], ignore_index=False)
                
            if str(subject)[-10:] == 'motion.csv':
                df_aux = pd.read_csv(os.path.join(path,subject))
                df_aux[subject[:-11]] = str(subject[:-11])
                df_aux = df_aux.set_index([subject[:-11],'File'], inplace=False)
                motion = pd.concat([df_aux,motion], ignore_index=False)
                      
        
        else:
            # Run LOOP
            for run in os.listdir(os.path.join(path,subject)):
                if run != '.DS_Store':
                    tuples = [(dataset,subject,run)]
                    teste = pd.read_csv(str(os.path.join(path,subject)) + '/' + run, sep='\t').drop(columns=["File", "Sub-brick"])
                    if teste.shape[0] != 0:
                        df = pd.concat([df,pd.DataFrame(remove_triangle(teste.corr(method='pearson')),index=pd.MultiIndex.from_tuples(tuples, names=['Institution', 'Subject', 'Run']))])
 
                                        #pd.DataFrame((remove_triangle(teste.corr(method='pearson'))), 
                                        #index=pd.MultiIndex.from_tuples(tuples, names=['Institution', 'Subject', 'Run']))])

    end = time.time()
    print((end - start)/60)
    
phenotypic.index.names=['Institution','Subject']
motion.index.names=['Institution','File']
phenotypic.to_csv(r'/Users/rodrigo/Post-Grad/CC400/phenotypic200.csv', header=True)
motion.to_csv(r'/Users/rodrigo/Post-Grad/CC400/motion200.csv', header=True)
df.to_csv(r'/Users/rodrigo/Post-Grad/CC400/corr_matrices200.csv', header=True)

KKI
1018959
1019436
1043241
1266183
1535233
1541812
1577042
1594156
1623716
1638334
1652369
1686265
1692275
1735881
1779922
1842819
1846346
1873761
1962503
1988015
1996183
2014113
2018106
2026113
2081148
2104012
2138826
2299519
2344857
2360428
2371032
2554127
2558999
2572285
2601925
2618929
2621228
2640795
2641332
2703289
2740232
2768273
2822304
2903997
2917777
2930625
3103809
3119327
3154996
3160561
3170319
3310328
3434578
3486975
3519022
3611827
3699991
3713230
3813783
3884955
3902469
3912996
3917422
3972472
3972956
4104523
4154182
4275075
4362730
4601682
5216908
6346605
6453038
7129258
7415617
7774305
8083695
8263351
8337695
8432725
8628223
8658218
9922944
KKI_motion.csv
KKI_phenotypic.csv
0.037899716695149736
NYU
0010002
0010003
0010004
0010005
0010006
0010007
0010008
0010009
0010010
0010011
0010012
0010013
0010014
0010015
0010016
0010017
0010018
0010019
0010020
0010021
0010022
0010023
0010024
0010025
0010026
0010027
0010028
0010029
0010030
0010031
0010032
0010033
0010034
0010035
0

## Validation 

In [68]:
phenotypic

Site  Gender    Age  Handedness  DX   Secondary Dx   \
Institution Subject                                                        
WashU       15001       8     1.0   9.94         1.0   0             NaN   
            15002       8     0.0   7.17         1.0   0             NaN   
            15003       8     1.0   9.18         1.0   0             NaN   
            15004       8     0.0   7.46         1.0   0             NaN   
            15005       8     0.0  21.13         1.0   0             NaN   
...                   ...     ...    ...         ...  ..             ...   
KKI         8337695     3     1.0   8.80         1.0   1             ODD   
            8432725     3     0.0  11.51         1.0   0   Simple Phobia   
            8628223     3     0.0  10.81         1.0   0   Simple phobia   
            8658218     3     1.0   9.75         1.0   0  Simple Phobia    
            9922944     3     1.0  10.30         1.0   0             NaN   

                     ADHD Measure  ADHD Index  Inattentive  Hyper/Impulsive  \
Institution Subject                                                           
WashU       15001             NaN         NaN          NaN              NaN   
            15002             NaN         NaN          NaN              NaN   
            15003             NaN         NaN          NaN              NaN   
            15004             NaN         NaN          NaN              NaN   
            15005             NaN         NaN          NaN              NaN   
...                           ...         ...          ...              ...   
KKI         8337695           2.0        71.0         74.0             82.0   
            8432725           2.0      -999.0       -999.0           -999.0   
            8628223           2.0        42.0         49.0             49.0   
            8658218           2.0        56.0         55.0             52.0   
            9922944           2.0        49.0         50.0             54.0   

                     ...  QC_S1_Anat  QC_S2_Rest_1  QC_S2_Rest_2  QC_S2_Anat  \
Institution Subject  ...                                                       
WashU       15001    ...         0.0           NaN           NaN         NaN   
            15002    ...         1.0           NaN           NaN         NaN   
            15003    ...         1.0           NaN           NaN         NaN   
            15004    ...         1.0           NaN           NaN         NaN   
            15005    ...         1.0           1.0           1.0         NaN   
...                  ...         ...           ...           ...         ...   
KKI         8337695  ...         NaN           NaN           NaN         NaN   
            8432725  ...         NaN           NaN           NaN         NaN   
            8628223  ...         NaN           NaN           NaN         NaN   
            8658218  ...         NaN           NaN           NaN         NaN   
            9922944  ...         NaN           NaN           NaN         NaN   

                     QC_Rest_1  QC_Rest_2  QC_Rest_3  QC_Rest_4  \
Institution Subject                                               
WashU       15001          NaN        NaN        NaN        NaN   
            15002          NaN        NaN        NaN        NaN   
            15003          NaN        NaN        NaN        NaN   
            15004          NaN        NaN        NaN        NaN   
            15005          NaN        NaN        NaN        NaN   
...                        ...        ...        ...        ...   
KKI         8337695        1.0        NaN        NaN        NaN   
            8432725        1.0        NaN        NaN        NaN   
            8628223        1.0        NaN        NaN        NaN   
            8658218        0.0        NaN        NaN        NaN   
            9922944        1.0        NaN        NaN        NaN   

                     QC_Anatomical_1  QC_Anatomical_2  
Institution Subject                

In [69]:
#df = df.dropna(axis=0)
df

0      \
Institution Subject Run                                                       
KKI         1018959 snwmrda1018959_session_1_rest_1_cc200_TCs.1D   0.062784   
                    sfnwmrda1018959_session_1_rest_1_cc200_TCs.1D  0.082767   
            1019436 sfnwmrda1019436_session_1_rest_1_cc200_TCs.1D  0.216872   
                    snwmrda1019436_session_1_rest_1_cc200_TCs.1D   0.182301   
            1043241 sfnwmrda1043241_session_1_rest_1_cc200_TCs.1D -0.060757   
...                                                                     ...   
WashU       0015062 snwmrda0015062_session_1_rest_3_cc200_TCs.1D  -0.021718   
                    snwmrda0015062_session_1_rest_2_cc200_TCs.1D  -0.180944   
                    sfnwmrda0015062_session_1_rest_2_cc200_TCs.1D -0.134401   
                    sfnwmrda0015062_session_1_rest_1_cc200_TCs.1D -0.064805   
                    snwmrda0015062_session_1_rest_1_cc200_TCs.1D   0.026965   

                                                                      1      \
Institution Subject Run                                                       
KKI         1018959 snwmrda1018959_session_1_rest_1_cc200_TCs.1D  -0.299914   
                    sfnwmrda1018959_session_1_rest_1_cc200_TCs.1D -0.202121   
            1019436 sfnwmrda1019436_session_1_rest_1_cc200_TCs.1D -0.055456   
                    snwmrda1019436_session_1_rest_1_cc200_TCs.1D   0.021330   
            1043241 sfnwmrda1043241_session_1_rest_1_cc200_TCs.1D  0.218841   
...                                                                     ...   
WashU       0015062 snwmrda0015062_session_1_rest_3_cc200_TCs.1D   0.125678   
                    snwmrda0015062_session_1_rest_2_cc200_TCs.1D   0.140382   
                    sfnwmrda0015062_session_1_rest_2_cc200_TCs.1D  0.144484   
                    sfnwmrda0015062_session_1_rest_1_cc200_TCs.1D -0.124670   
                    snwmrda0015062_session_1_rest_1_cc200_TCs.1D  -0.062537   

                                                                      2      \
Institution Subject Run                                                       
KKI         1018959 snwmrda1018959_session_1_rest_1_cc200_TCs.1D  -0.115178   
                    sfnwmrda1018959_session_1_rest_1_cc200_TCs.1D -0.253291   
            1019436 sfnwmrda1019436_session_1_rest_1_cc200_TCs.1D  0.274632   
                    snwmrda1019436_session_1_rest_1_cc200_TCs.1D   0.220501   
            1043241 sfnwmrda1043241_session_1_rest_1_cc200_TCs.1D -0.220541   
...                                                                     ...   
WashU       0015062 snwmrda0015062_session_1_rest_3_cc200_TCs.1D  -0.249868   
                    snwmrda0015062_session_1_rest_2_cc200_TCs.1D  -0.124513   
                    sfnwmrda0015062_session_1_rest_2_cc200_TCs.1D -0.073773   
                    sfnwmrda0015062_session_1_rest_1_cc200_TCs.1D -0.040828   
                    snwmrda0015062_session_1_rest_1_cc200_TCs.1D  -0.109554   

                                                                      3      \
Institution Subject Run                                                       
KKI         1018959 snwmrda1018959_session_1_rest_1_cc200_TCs.1D   0.041176   
                    sfnwmrda1018959_session_1_rest_1_cc200_TCs.1D  0.143162   
            1019436 sfnwmrda1019436_session_1_rest_1_cc200_TCs.1D  0.057173   
                    snwmrda1019436_session_1_rest_1_cc200_TCs.1D   0.112301   
            1043241 sfnwmrda1043241_session_1_rest_1_cc200_TCs.1D -0.009787   
...                                                                     ...   
WashU       0015062 snwmrda0015062_session_1_rest_3_cc200_TCs.1D   0.050053   
                    snwmrda0015062_session_1_rest_2_cc200_TCs.1D  -0.030457   
                    sfnwmrda0015062_session_1_rest_2_cc200_TCs.1D -0.070069   
                    sfnwmrda0015062_session_1_rest_1_cc200_TCs.1D  0.036085   
                    snwmrda0015062_ses

In [70]:
motion

Max Motion (mm)  \
Institution File                                              
WashU       rp_0015001_session_4_rest_1.1D            0.422   
            rp_0015001_session_4_rest_2.1D            0.469   
            rp_0015001_session_4_rest_3.1D            0.412   
            rp_0015001_session_4_rest_4.1D            0.645   
            rp_0015001_session_4_rest_5.1D            0.875   
...                                                     ...   
KKI         rp_8337695_session_1_rest_1.1D            2.362   
            rp_8432725_session_1_rest_1.1D            1.638   
            rp_8628223_session_1_rest_1.1D            1.483   
            rp_8658218_session_1_rest_1.1D            2.530   
            rp_9922944_session_1_rest_1.1D            1.142   

                                            Max Motion Time Point  \
Institution File                                                    
WashU       rp_0015001_session_4_rest_1.1D                     45   
            rp_0015001_session_4_rest_2.1D                     21   
            rp_0015001_session_4_rest_3.1D                     38   
            rp_0015001_session_4_rest_4.1D                     23   
            rp_0015001_session_4_rest_5.1D                     71   
...                                                           ...   
KKI         rp_8337695_session_1_rest_1.1D                    104   
            rp_8432725_session_1_rest_1.1D                     54   
            rp_8628223_session_1_rest_1.1D                     98   
            rp_8658218_session_1_rest_1.1D                    118   
            rp_9922944_session_1_rest_1.1D                    148   

                                             Max Rotation (degree)  \
Institution File                                                     
WashU       rp_0015001_session_4_rest_1.1D                   0.254   
            rp_0015001_session_4_rest_2.1D                   0.250   
            rp_0015001_session_4_rest_3.1D                   2.098   
            rp_0015001_session_4_rest_4.1D                   0.560   
            rp_0015001_session_4_rest_5.1D                   0.811   
...                                                            ...   
KKI         rp_8337695_session_1_rest_1.1D                   3.065   
            rp_8432725_session_1_rest_1.1D                   4.338   
            rp_8628223_session_1_rest_1.1D                   0.830   
            rp_8658218_session_1_rest_1.1D                   5.735   
            rp_9922944_session_1_rest_1.1D                   0.742   

                                             Max Rotation Time Point  \
Institution File                                                       
WashU       rp_0015001_session_4_rest_1.1D                        62   
            rp_0015001_session_4_rest_2.1D                        57   
            rp_0015001_session_4_rest_3.1D                        35   
            rp_0015001_session_4_rest_4.1D                         7   
            rp_0015001_session_4_rest_5.1D                        32   
...                                                              ...   
KKI         rp_8337695_session_1_rest_1.1D                       125   
            rp_8432725_session_1_rest_1.1D                        54   
            rp_8628223_session_1_rest_1.1D                       148   
            rp_8658218_session_1_rest_1.1D                       118   
            rp_9922944_session_1_rest_1.1D                       108   

                                             Max X (mm)   Max Y (mm)  \
Institution File                                                       
WashU       rp_0015001_session_4_rest_1.1D        0.071       -0.184   
            rp_0015001_session_4_rest_2.1D       -0.121       -0.055   
            rp_0015001_session_4_rest_3.1D       -0.208        0.171   
            rp_0015001_session_4_rest_4.1D        0.195       -0.005   
            rp_0015001_session_4_rest_5.1D    

In [71]:
df_aux

Site  Gender    Age  Handedness  DX  Secondary Dx   \
WashU ScanDir ID                                                       
WashU 15001          8       1   9.94           1   0            NaN   
      15002          8       0   7.17           1   0            NaN   
      15003          8       1   9.18           1   0            NaN   
      15004          8       0   7.46           1   0            NaN   
      15005          8       0  21.13           1   0            NaN   
...                ...     ...    ...         ...  ..            ...   
      15058          8       0  14.08           1   0            NaN   
      15059          8       0   9.05           1   0            NaN   
      15060          8       1   9.76           1   0            NaN   
      15061          8       1  12.04           1   0            NaN   
      15062          8       1  12.88           1   0            NaN   

                  ADHD Measure  ADHD Index  Inattentive  Hyper/Impulsive  ...  \
WashU ScanDir ID                                                          ...   
WashU 15001                NaN         NaN          NaN              NaN  ...   
      15002                NaN         NaN          NaN              NaN  ...   
      15003                NaN         NaN          NaN              NaN  ...   
      15004                NaN         NaN          NaN              NaN  ...   
      15005                NaN         NaN          NaN              NaN  ...   
...                        ...         ...          ...              ...  ...   
      15058                NaN         NaN          NaN              NaN  ...   
      15059                NaN         NaN          NaN              NaN  ...   
      15060                NaN         NaN          NaN              NaN  ...   
      15061                NaN         NaN          NaN              NaN  ...   
      15062                NaN         NaN          NaN              NaN  ...   

                  QC_S1_Rest_1  QC_S1_Rest_2  QC_S1_Rest_3  QC_S1_Rest_4  \
WashU ScanDir ID                                                           
WashU 15001                1.0           1.0           1.0           0.0   
      15002                1.0           0.0           0.0           0.0   
      15003                1.0           0.0           0.0           0.0   
      15004                0.0           0.0           0.0           1.0   
      15005                1.0           1.0           NaN           NaN   
...                        ...           ...           ...           ...   
      15058                1.0           1.0           1.0           NaN   
      15059                0.0           1.0           0.0           NaN   
      15060                0.0           0.0           0.0           NaN   
      15061                0.0           0.0           0.0           NaN   
      15062                1.0           1.0           1.0           NaN   

                  QC_S1_Rest_5  QC_S1_Rest_6  QC_S1_Anat  QC_S2_Rest_1  \
WashU ScanDir ID                                                         
WashU 15001                0.0           0.0         0.0           NaN   
      15002                0.0           0.0         1.0           NaN   
      15003                0.0           0.0         1.0           NaN   
      15004                1.0           0.0         1.0           NaN   
      15005                NaN           NaN         1.0           1.0   
...                        ...           ...         ...           ...   
      15058                NaN           NaN         1.0           NaN   
      15059                NaN           NaN         1.0           NaN   
      15060                NaN           NaN         1.0           NaN   
      15061                NaN           NaN         1.0           NaN   
      15062                NaN           NaN         1.0           NaN   

                  QC_S2_Rest_2  QC_S2_Anat  
WashU ScanDir ID              